# Advanced Catalog Product Operations

The `Descartes Labs Catalog API` is a single interface through which you can discover existing raster `Products`, search and retrieve their associated `Images`, and manage your own `Products`.

This guide is meant to serve as an introduction to miscellaneous advanced concepts and operations with `Descartes Labs Catalog Products`. For a more in depth overview of all of `Catalog's` classes and their capabilities please visit the [API Reference](https://docs.descarteslabs.com/descarteslabs/catalog/readme.html) and [Catalog Guide](https://docs.descarteslabs.com/guides/catalog.html) sections in our Documentation page.

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import Product, properties as p

In [ ]:
import numpy as np

For this guide we will search several `Products` over the same `DLTile`:

In [ ]:
dltile = dl.geo.DLTile.from_latlon(
    46.63653, 7.93202, resolution=10.0, tilesize=1024, pad=0
)
dltile

## Bitmask QA Bands
Some `Catalog Products` contain various quality flags within a single band, otherwise known as a `bitmask`. Below is an example of working with the `Sentinel-2 L2A SCL band` to retrieve information contained within this bitmask. 

First, we'll retrieve the `SCL` band as a `numpy ndarray`. Note that the returned `data_type` for this band is `Unsigned 8-bit integer`, or internally as `dl.catalog.DataType.BYTE`

In [ ]:
s2_product = Product.get("esa:sentinel-2:l2a:v1")
s2_images = s2_product.images()
s2_ic = (
    s2_images.intersects(dltile)
    .filter("2023-10-01" < p.acquired < "2023-10-05")
    .filter(p.cloud_fraction < 0.1)
).collect()
s2_ic

In [ ]:
scl_arr = s2_ic.mosaic(bands=["scl"])
rgb_arr = s2_ic.mosaic(bands=["red", "green", "blue"])
print(f"RGB Shape: {rgb_arr.shape}")
print(f"RGB DType: {rgb_arr.dtype}")
print(f"SCL Shape: {scl_arr.shape}")
print(f"DType: {scl_arr.dtype}")
dl.utils.display(*(rgb_arr, scl_arr), title=["RGB", "SCL Integer Values"], size=5)

That the `integer` values themselves do not represent anything meaningful, however converting them to `bits` reveals a wealth of information through a series of `boolean` flags. 

While we do calculate much of the `Sentinel-2 L2A SCL` information into its own separate band, there are a handful of classifications we do not. [More information on what else is contained within the Sentinel-2 L2A SCL band](https://sentinels.copernicus.eu/documents/247904/446933/Sentinel-2-Level-2A-Algorithm-Theoretical-Basis-Document-ATBD.pdf/fe5bacb4-7d4c-9212-8606-6591384390c3?t=1643102691874)

Here we will define a simple function which converts the returned `integer` values into `bits`, then check the specified `bit position` for the presence of dark area pixels or cast shadows. 

In [ ]:
# Function to return individual mask from QA bits
def bit_mask(array, bit_pos, num_bits, cond_val):
    """
    array -> numpy array
    bit_pos -> bit position (index in to bit string)
    num_bits -> number of bits occuppied by bit flag
    cond_val -> conditional value (boolean 0,1)
    """
    pos_value = num_bits << bit_pos
    con_value = cond_val << bit_pos
    return (array & pos_value) == con_value

When we invoke this function, each pixel is evaluated `True` if it matches our filter condition and `False` if not:

In [ ]:
cast_shadow = bit_mask(scl_arr, 2, 1, 0)
cast_shadow

Lastly, we can `mask` our RGB data to our newly calculated mask:

In [ ]:
mask_3d = np.broadcast_to(cast_shadow, rgb_arr.shape)
rgb_mask = np.ma.masked_array(data=rgb_arr, mask=mask_3d)
dl.utils.display(*(rgb_arr, rgb_mask), title=["RGB", "RGB Masked"], size=5)

## Filtering Ascending and Descendng Sentinel-1 Collections
Oftentimes we want to apply an arbitrary `filter` against a collection of objects. 

In this example we will filter an `ImageCollection` of `Sentinel-1` images for ascending orbit.

In [ ]:
s1_product = Product.get("esa:sentinel-1:sigma0v:v1")
s1_images = s1_product.images()
s1_ic = (
    s1_images.intersects(dltile).filter("2023-01-01" <= p.acquired < "2023-02-01")
).collect()
s1_ic

Filtering function to be applied in an `ImageCollection.filter()`. Note this information is stored in the `Image ID` and the evaluation is a simple `.endswith()`:

In [ ]:
def filter_asc(s):
    sid = s.id
    ascending = sid.split("-")[-2].endswith("A")
    return ascending

Apply our filter:

In [ ]:
asc_ic = s1_ic.filter(filter_asc)
asc_ic